In [5]:
import numpy as np
import pandas as pd
import json
import csv
import random

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

import pprint
import tensorflow.compat.v1 as tf
from tensorflow.python.framework import ops
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
tf.disable_eager_execution()

# Reading the data
data = pd.read_csv('D:/7 sem/nlp - projects/News_2.csv')
data.head()


,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [6]:
data = data.drop(["Unnamed: 0"], axis=1)
data.head(5)


,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [7]:
# encoding the labels
le = preprocessing.LabelEncoder()
le.fit(data['label'])
data['label'] = le.transform(data['label'])

In [8]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1

In [9]:
title = []
text = []
labels = []
for x in range(training_size):
    title.append(data['title'][x])
    text.append(data['text'][x])
    labels.append(data['label'][x])

In [10]:
tokenizer1 = Tokenizer()
tokenizer1.fit_on_texts(title)
word_index1 = tokenizer1.word_index
vocab_size1 = len(word_index1)
sequences1 = tokenizer1.texts_to_sequences(title)
padded1 = pad_sequences(
    sequences1, padding=padding_type, truncating=trunc_type)
split = int(test_portion * training_size)
training_sequences1 = padded1[split:training_size]
test_sequences1 = padded1[0:split]
test_labels = labels[0:split]
training_labels = labels[split:training_size]

In [11]:
embeddings_index = {}

with open('D:/7 sem/nlp - projects/glove.6B.50d.txt', encoding='utf-8') as f:   
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Generating embeddings
embeddings_matrix = np.zeros((vocab_size1+1, embedding_dim))
for word, i in word_index1.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector


In [12]:
model = tf.keras.Sequential([
tf.keras.layers.Embedding(vocab_size1+1, embedding_dim,
                            input_length=max_length, weights=[
                                embeddings_matrix],
                            trainable=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv1D(64, 5, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=4),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
        optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 54, 50)            377600    
                                                                 
 dropout (Dropout)           (None, 54, 50)            0         
                                                                 
 conv1d (Conv1D)             (None, 50, 64)            16064     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 64)           0         
 )                                                               
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                        

In [13]:
num_epochs = 50

# Assuming you have prepared your sequences and labels
training_padded = pad_sequences(training_sequences1, maxlen=54, padding='post')
training_labels = np.array(training_labels)
testing_padded = pad_sequences(test_sequences1, maxlen=54, padding='post')
testing_labels = np.array(test_labels)

history = model.fit(training_padded, training_labels,
                    epochs=num_epochs,
                    validation_data=(testing_padded, testing_labels),
                    verbose=2)

Train on 2700 samples, validate on 300 samples
Epoch 1/50


C:\Users\user\AppData\Roaming\Python\Python39\site-packages\keras\engine\training_v1.py:2332: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


2700/2700 - 4s - loss: 0.6497 - accuracy: 0.6044 - val_loss: 0.5831 - val_accuracy: 0.6833 - 4s/epoch - 1ms/sample
Epoch 2/50
2700/2700 - 2s - loss: 0.5683 - accuracy: 0.7037 - val_loss: 0.5418 - val_accuracy: 0.7133 - 2s/epoch - 617us/sample
Epoch 3/50
2700/2700 - 2s - loss: 0.5208 - accuracy: 0.7411 - val_loss: 0.5246 - val_accuracy: 0.7200 - 2s/epoch - 612us/sample
Epoch 4/50
2700/2700 - 1s - loss: 0.4970 - accuracy: 0.7493 - val_loss: 0.5342 - val_accuracy: 0.7100 - 1s/epoch - 549us/sample
Epoch 5/50
2700/2700 - 2s - loss: 0.4374 - accuracy: 0.7952 - val_loss: 0.5219 - val_accuracy: 0.7467 - 2s/epoch - 562us/sample
Epoch 6/50
2700/2700 - 2s - loss: 0.4000 - accuracy: 0.8174 - val_loss: 0.5371 - val_accuracy: 0.7267 - 2s/epoch - 634us/sample
Epoch 7/50
2700/2700 - 2s - loss: 0.3681 - accuracy: 0.8496 - val_loss: 0.4901 - val_accuracy: 0.7633 - 2s/epoch - 617us/sample
Epoch 8/50
2700/2700 - 2s - loss: 0.3456 - accuracy: 0.8507 - val_loss: 0.5246 - val_accuracy: 0.7433 - 2s/epoch - 58

In [14]:
# model.save('fake_news_trained_model.h5')

In [19]:
# sample text to check if fake or not
# New text for inference
X = "Kerry to go to Paris in gesture of sympathy"

# detection
sequences = tokenizer1.texts_to_sequences([X])[0]
sequences = pad_sequences([sequences], maxlen=54,
                        padding=padding_type,
                        truncating=trunc_type)
if(model.predict(sequences, verbose=0)[0][0] >= 0.5):
    print("This news is True")
else:
    print("This news is false")


This news is false
